In [52]:
import requests
import pandas as pd
# Librería que convierte la respuesta xml 
# de la api a un diccionario.
import xmltodict

In [56]:
indicador = 'SP.DYN.LE00.IN'
pais = 'all'
iyear = '2000'
fyear = '2002'
date = iyear + ':'+fyear
api_url = ' http://api.worldbank.org/V2'
path = f'/country/{pais}/indicator/{indicador}?date={iyear}:{fyear}&format=json&prefix=Getdata'
url = api_url + path
url

dict_data = requests.get(url).json()[1]
dict_data

[{'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2002',
  'value': 52.0431492826438,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2001',
  'value': 51.6064582838941,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2000',
  'value': 51.2761288844047,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZI', 'value': 'A

In [57]:
type(dict_data)

list

In [58]:
# Verifico la salida de uno de estos diccionarios 
# dentro de la lista de diccionarios
data = pd.DataFrame(dict_data[0])
data

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
id,SP.DYN.LE00.IN,ZH,AFE,2002,52.043149,,,0
value,"Life expectancy at birth, total (years)",Africa Eastern and Southern,AFE,2002,52.043149,,,0


In [61]:
# Mido el largo de la respuesta que es una lista
extención = len(dict_data)
extención

50

In [62]:
# sumo todos los pequeños diccionarios que se encuentran en la lista
sumatoria = [dict_data[i] for i in range(extención)]
data = pd.json_normalize(sumatoria)

In [63]:
data

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2002,52.043149,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
1,AFE,2001,51.606458,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
2,AFE,2000,51.276129,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
3,AFW,2002,49.816929,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZI,Africa Western and Central
4,AFW,2001,49.475136,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZI,Africa Western and Central
5,AFW,2000,49.219714,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZI,Africa Western and Central
6,ARB,2002,68.340246,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",1A,Arab World
7,ARB,2001,68.072519,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",1A,Arab World
8,ARB,2000,67.800591,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",1A,Arab World
9,CSS,2002,71.713363,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",S3,Caribbean small states
